In [105]:
import als_model

In [58]:
%matplotlib inline

In [72]:
import geocoder

In [75]:
run dataprep.py

Carkeek Warmer Carkeek Park, Seattle, WA <[OVER_QUERY_LIMIT] Google - Geocode [empty]> None


In [76]:
#manually add the one missing zipcode
g = geocoder.google("Carkeek Park, Seattle, WA")

In [77]:
g

<[OK] Google - Geocode [Carkeek Park, 950 NW Carkeek Park Rd, Seattle, WA 98177, USA]>

In [91]:
cleaned_df['Venue_Zip2'] = cleaned_df[['Venue_Zip', 'Event_Name']].apply(lambda row: 98177 if row[1]=='Carkeek Warmer'
                                                                       else row[0], axis=1)

In [94]:
#pickle the cleaned_df for future use without having to re-run dataprep.py 
#(this will override the version automatically saved) by dataprep.py with the one with the corrected zipcode
with open ('cleaned_df.pkl', 'w') as f:
    pickle.dump(cleaned_df, f)
    
#to load this file in the future, run command:
#with open('cleaned_df.pkl', 'rb') as f:
    #dataframe = pickle.load(f)

## Instantiate Class Instance for ALS Model, Prep and Run:

In [106]:
als_model = als_model.implicit_als(cleaned_df)

In [107]:
als_model.prep_spark_full_df()

DataFrame[PersonID: bigint, EventID: bigint, Participated: bigint, Event_Date: bigint, SeriesID: double, EventTypeID: bigint, Total_Fee_Avg: bigint, Miles2_Avg: bigint, Venue_Zip: bigint]

In [108]:
als_model.spark_full_df.show(5)

+--------+-------+------------+-------------------+--------+-----------+-------------+----------+---------+
|PersonID|EventID|Participated|         Event_Date|SeriesID|EventTypeID|Total_Fee_Avg|Miles2_Avg|Venue_Zip|
+--------+-------+------------+-------------------+--------+-----------+-------------+----------+---------+
|       1|     11|           1|1423958400000000000|     0.0|          1|           46|        12|    98239|
|       2|     11|           0|1423958400000000000|     0.0|          1|           46|        12|    98239|
|       3|     11|           0|1423958400000000000|     0.0|          1|           46|        12|    98239|
|       4|     11|           0|1423958400000000000|     0.0|          1|           46|        12|    98239|
|       5|     11|           0|1423958400000000000|     0.0|          1|           46|        12|    98239|
+--------+-------+------------+-------------------+--------+-----------+-------------+----------+---------+
only showing top 5 rows



In [109]:
als_model.train_val_test_split()

Train Size: 592450.0
Validation Size: 148113.0
Test Size: 185141.0


In [110]:
als_model.print_train_val_test_info("EventID")

participants in train: 10764
participants in validate: 10764
participants in test: 10764


participants in both train & validate: 10764
participants in both train & test: 10764


EventID in train: 56
EventID in validate: 14
EventID in test: 18


EventID in both train & validate: 1
EventID in both train & test: 0


In [111]:
als_model.create_participate_matrices("EventID")

In [112]:
als_model.run_ALS_TVS(event_param="EventID", scoring="r2")

Predictions on Training Data:
Predictions includes 592450 valid values and 0 nan values


Mean prediction is 0.0303024963926
Error of Type r2 = 0.317162022448
Predictions on Validation Data:
Predictions includes 10334 valid values and 137779 nan values


Mean prediction is 0.00201103141265
Error of Type r2 = -0.0142867802634


In [128]:
parammap = als_model.tvs_model.getEstimatorParamMaps()
avgmetrics = als_model.tvs_model.validationMetrics

zipped_tvs_model_info = zip(parammap, avgmetrics)

In [135]:
min(avgmetrics), max(avgmetrics)
#All combinations of parameters give the same validation number...how can this be?

(-0.14203634369376705, -0.14203634369376705)

In [140]:
full_df_pd = als_model.spark_full_df.toPandas()
als_fitted_model_tvsr2 = als_model.tvs_bestmodel
als_trainpreds_tvsr2_pd = als_model.tvs_trainpreds.toPandas()

with open ("24OCT17_pipelinerun/full_df_pd.pkl", 'w') as f:
    pickle.dump(full_df_pd, f)

als_fitted_model_tvsr2.save("24OCT17_pipelinerun/als_fitted_model_tvsr2")

with open ("24OCT17_pipelinerun/als_trainpreds_tvsr2.pkl", 'w') as f:
    pickle.dump(als_trainpreds_tvsr2_pd, f)

#Note: to load the model saved above in future, run command:
#sameModel = GradientBoostedTreesModel.load("tvs_bestmodel_base")

In [144]:
import prep_gbdata

In [161]:
gb_dataprep = prep_gbdata.RegressionDataPrep(als_model.tvs_trainpreds, als_model.spark_full_df, user_df=cleaned_df)

In [162]:
gb_dataprep.format_gb_data()

In [163]:
gb_dataprep.train_gb.head()

,Event_Date,Total_Fee_Avg,Miles2_Avg,AgeAvg,y_label,SeriesID_1.0,SeriesID_2.0,SeriesID_3.0,SeriesID_4.0,EventTypeID_2,...,Venue_Zip_98118,Venue_Zip_98177,Venue_Zip_98208,Venue_Zip_98239,Venue_Zip_98290,Venue_Zip_98332,Venue_Zip_98922,Venue_Zip_99032,Gender_Male,Gender_Other
0,1423958400000000000,46,12,28,1.000000,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
1,1423958400000000000,46,12,60,0.021625,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,1423958400000000000,46,12,40,0.000000,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
3,1423958400000000000,46,12,40,0.017983,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
4,1423958400000000000,46,12,58,0.051031,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
